# **Стандартизация векторов и матрица корреляций**

***
## **СТАНДАРТИЗАЦИЯ ВЕКТОРОВ**

В модулях по разведывательному анализу данных и машинному обучению мы не раз говорили о преобразованиях признаков путём нормализации и стандартизации. Вспомним, что это такое
***
* **Нормализация** — это процесс приведения признаков к единому масштабу, например от 0 до 1. Пример — min-max-нормализация:

![](data/31.PNG)

* **Стандартизация** — это процесс приведения признаков к единому масштабу характеристик распределения — нулевому среднему и единичному стандартному отклонению:

![](data/32.PNG)
***

В линейной алгебре под стандартизацией вектора ![](data/33.PNG) понимается несколько другая операция, которая проходит в два этапа:

1. **Центрирование вектора** — это операция приведения среднего к 0:

![](data/34.PNG)

2. **Нормирование вектора** — это операция приведения диапазона вектора к масштабу от -1 до 1 путём деления центрированного вектора на его длину:

![](data/35.PNG)

В результате стандартизации вектора всегда получается новый вектор, длина которого равна 1:

![](data/36.PNG)

***

![](data/37.PNG)  
![](data/38.PNG)

Как видите, теперь оба признака имеют значения от -1 до 1 и равный порядок, в отличие от исходных признаков.
***
Давайте посмотрим, что произойдёт с матрицей Грама после стандартизации векторов x1 и x2:

![](data/39.PNG)

Как видите, все числа — в диапазоне от -1 до 1. 
***
Забегая вперёд, скажем, что это так называемые **выборочные корреляции признаков**, а сама матрица является **матрицей корреляций** или **корреляционной матрицей**. Пока просто запомните, как выглядит эта матрица.

![](data/40.PNG)

* После стандартизации мы прогоняем регрессию стандартизованного y на стандартизованные регрессоры **без константы**.
***

**В ЧЁМ БОНУСЫ?**

Математика говорит, что регрессия исходного y на исходные («сырые») признаки c константой точно такая же, как регрессия стандартизированного на стандартизированные признаки без константы. В чём же разница? Математически — ни в чём.

На прогноз модели линейной регрессии, построенной по МНК, и её качество стандартизация практически не влияет. Масштабы признаков будут иметь значение только в том случае, если для поиска коэффициентов вы используете численные методы, такие как градиентный спуск (SGDRegressor из sklearn). О нём мы поговорим, когда будем знакомиться с алгоритмом градиентного спуска в модуле по оптимизации.

Однако с точки зрения интерпретации важности коэффициентов разница есть. Если вы занимаетесь отбором наиболее важных признаков по значению коэффициентов линейной регрессии на нестандартизированных данных, это будет не совсем корректно: один признак может изменяться от 0 до 1, а второй — от -1000 до 1000. Коэффициенты при них также будут различного масштаба. Если же вы посмотрите оценки коэффициентов регрессии после стандартизации, то они будут в едином масштабе, что даст более цельную и объективную картину.

Более важный бонус заключается в том, что после стандартизации матрица Грама признаков как по волшебству превращается в корреляционную матрицу, о которой пойдёт речь далее. Почему это хорошо? На свойства корреляционной матрицы опираются такие алгоритмы, как метод главных компонент и сингулярное разложение, а так как «сырая» и стандартизированная регрессия математически эквивалентны, то имеет смысл исследовать стандартизированную, а результаты обобщить на «сырую».

### Рассмотрим всё это на примере ↓

Пример № 3

Вновь рассмотрим данные о стоимости жилья в районах Бостона.

На этот раз возьмём четыре признака: CHAS, LSTAT, CRIM и RM.

Для начала посмотрим на статистические характеристики с помощью метода describe():

In [1]:
# Загрузка библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
from sklearn import datasets # для импорта данных
import seaborn as sns # для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков

# загружаем датасет
boston = datasets.load_boston()
boston_data = pd.DataFrame(
    data=boston.data, #данные
    columns=boston.feature_names #наименования столбцов
)
boston_data['PRICE'] = boston.target
boston_data.head()

c:\Users\Roman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [2]:
boston_data[['CHAS', 'LSTAT', 'CRIM','RM']].describe()

,CHAS,LSTAT,CRIM,RM
count,506.000000,506.000000,506.000000,506.000000
mean,0.069170,12.653063,3.613524,6.284634
std,0.253994,7.141062,8.601545,0.702617
min,0.000000,1.730000,0.006320,3.561000
25%,0.000000,6.950000,0.082045,5.885500
50%,0.000000,11.360000,0.256510,6.208500
75%,0.000000,16.955000,3.677083,6.623500
max,1.000000,37.970000,88.976200,8.780000


Видим, что каждый из признаков измеряется в различных единицах и изменяется в различных диапазонах: например, CHAS лежит в диапазоне от 0 до 1, а вот CRIM — в диапазоне от 0.006 до 88.976.

Рассмотрим модель линейной регрессии по МНК без стандартизации. Помним, что необходимо добавить столбец из единиц:


In [3]:
# составляем матрицу наблюдений и вектор целевой переменной
A = np.column_stack((np.ones(506), boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]))
y = boston_data[['PRICE']]
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

[[-1.92052548]
 [ 3.9975594 ]
 [-0.58240212]
 [-0.09739445]
 [ 5.07554248]]


Вот наши коэффициенты. Округлим их для наглядности:

![](data/41.PNG)

Давайте вспомним интерпретацию коэффициентов построенной модели линейной регрессии, которую мы изучали в модуле «ML-2. Обучение с учителем: регрессия». Значение коэффициента wi означает, на сколько в среднем изменится медианная цена (в тысячах долларов) при увеличении xi на 1.

Например, если количество низкостатусного населения (LSTAT) увеличится на 1 %, то медианная цена домов в районе (в среднем) упадёт на 0.6 тысяч долларов. А если среднее количество комнат (RM) в районе станет больше на 1, то медианная стоимость домов в районе (в среднем) увеличится на 5 тысяч долларов. 
***
*Тут в голову может прийти мысль: судя по значению коэффициентов, количество комнат (RM) оказывает на стоимость жилья большее влияние, чем процент низкостатусного населения (LSTAT). Однако такой вывод будет ошибочным. Мы не учитываем, что **признаки, а значит и коэффициенты линейной регрессии, лежат в разных масштабах**. Чтобы говорить о важности влияния признаков на модель, нужно строить её на стандартизированных данных.*
***
**Помним, что для построения стандартизированной линейной регрессии нам не нужен вектор свободных коэффициентов, а значит и столбец из единиц тоже не понадобится.**

Сначала центрируем векторы, которые находятся в столбцах матрицы A. Для этого вычтем среднее, вычисленное по строкам матрицы A в каждом столбце, с помощью метода mean(). Затем разделим результат на длины центрированных векторов, вычисленных с помощью функции linalg.norm().

Примечание. Обратите внимание, что для функции linalg.norm() обязательно необходимо указать параметр axis=0, так как по умолчанию норма считается для всей матрицы, а не для каждого столбца в отдельности. С определением нормы матрицы и тем, как она считается, вы можете ознакомиться в [**документации к функции norm().**](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)

In [4]:
# составляем матрицу наблюдений без дополнительного столбца из единиц
A = boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]
y = boston_data[['PRICE']]
# стандартизируем векторы в столбцах матрицы A
A_cent = A - A.mean()
A_st = A_cent/np.linalg.norm(A_cent, axis=0)
A_st.describe().round(2)

,CHAS,LSTAT,CRIM,RM
count,506.00,506.00,506.00,506.00
mean,-0.00,-0.00,-0.00,-0.00
std,0.04,0.04,0.04,0.04
min,-0.01,-0.07,-0.02,-0.17
25%,-0.01,-0.04,-0.02,-0.03
50%,-0.01,-0.01,-0.02,-0.00
75%,-0.01,0.03,0.00,0.02
max,0.16,0.16,0.44,0.16


Теперь векторы имеют одинаковые средние значения и стандартные отклонения. Если вычислить длину каждого из векторов, мы увидим, что они будут равны 1:

In [5]:
print(np.linalg.norm(A_st, axis=0))

[1. 1. 1. 1.]


Для получения стандартизированных коэффициентов нам также понадобится стандартизация целевой переменной  по тому же принципу:

In [6]:
# стандартизируем вектор целевой переменной
y_cent = y - y.mean()
y_st = y_cent/np.linalg.norm(y_cent)

Формула для вычисления коэффициента та же, что и раньше, только матрица A теперь заменяется на Ast, а y — на yst:

In [7]:
# вычислим OLS-оценку для стандартизированных коэффициентов
w_hat_st=np.linalg.inv(A_st.T@A_st)@A_st.T@y_st
print(w_hat_st.values)

[[ 0.11039956]
 [-0.45220423]
 [-0.09108766]
 [ 0.38774848]]


Вновь смотрим на коэффициенты. Помним, что коэффициента w0 у нас больше нет:

![](data/42.PNG)

Однако теперь интерпретировать сами коэффициенты в тех же измерениях у нас не получится.
***
**Сделаем важный вывод ↓**

Для того чтобы проинтерпретировать оценки коэффициентов линейной регрессии (**понять, каков будет прирост целевой переменной при изменении фактора на 1 условную единицу**), нам достаточно построить линейную регрессию в обычном виде без стандартизации и получить обычный вектор w.

Однако, чтобы корректно говорить о том, **какой фактор оказывает на прогноз большее влияние**, необходимо рассматривать стандартизированную оценку вектора коэффициентов wst.
***

In [8]:
# Давайте поближе взглянем на матрицу Грама для стандартизированных факторов:

# матрица Грама
A_st.T @ A_st

,CHAS,LSTAT,CRIM,RM
CHAS,1.000000,-0.053929,-0.055892,0.091251
LSTAT,-0.053929,1.000000,0.455621,-0.613808
CRIM,-0.055892,0.455621,1.000000,-0.219247
RM,0.091251,-0.613808,-0.219247,1.000000


На самом деле мы с вами только что вычислили **матрицу выборочных корреляций** наших исходных факторов. Мы уже сталкивались с ней много раз в разделах по разведывательному анализу данных и машинному обучению, правда, вычисляли её мы с помощью функции Pandas, а теперь научились делать это вручную.

Примечание. Матрицу корреляций можно получить только в том случае, если производить стандартизацию признаков как векторы (делить на длину центрированного вектора xst). Другие способы стандартизации/нормализации признаков не превращают матрицу Грама в матрицу корреляций.

In [11]:
x = np.array([12,8])

In [13]:
# Стандартизируйте вектор x = (12,8), приведя его к единичной длине.
# В качестве ответа введите координаты полученного вектора.
# Ответ округлите до третьего знака после точки-разделителя.

x = np.array([12,8])

x_cent = x - x.mean()
x_st = (x_cent/np.linalg.norm(x_cent)).round(3)
x_st

array([ 0.707, -0.707])

***
## **КОРРЕЛЯЦИОННАЯ МАТРИЦА**

* Напомним, что **корреляционная матрица** ***C*** — это матрица выборочных корреляций между факторами регрессий.
***
![](data/43.PNG)

Корреляция является одной из важнейших статистических характеристик выборки. Как мы уже знаем из модуля «EDA-2. Математическая статистика в контексте EDA», корреляцию можно измерять различным способами:

* корреляцией **Пирсона**;
* корреляцией **Спирмена**;
* корреляцией **Кендалла**.

В этом модуле мы будем говорить именно о **корреляции Пирсона**. Она измеряет тесноту линейных связей между непрерывными числовыми факторами и может принимать значения от -1 до +1.

![](data/44.PNG)

Как и любая статистическая величина, корреляция бывает **генеральной** и **выборочной**. Разница очень тонкая, и мы подробнее разберём её в модуле по теории вероятностей.
***
* **Генеральная (истинная) корреляция** — это теоретическая величина, которая отражает общую линейную зависимость между случайными величинами Xi и Xj. Забегая вперёд скажем, что данная характеристика является абстрактной и вычисляется для **генеральных совокупностей** — всех возможных реализаций Xi и Xj. В природе такой величины не существует, она есть только в теории вероятностей.
* **Выборочная корреляция** — это корреляция, вычисленная на ограниченной выборке. Это уже ближе к нашей теме. Выборочная корреляция отражает линейную взаимосвязь между факторами xi и xj, реализации которых представлены в выборке.
***

Выборочная корреляция между факторами высчитывается по громоздкой (на первый взгляд) формуле:

![](https://lms.skillfactory.ru/assets/courseware/v1/d4967fe68e49e8466304c6b2a1e5fc14/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_14.png)

Из вычисленных cij как раз и составляется матрица корреляций C. Если факторов k штук, то матрица C будет квадратной размера dimC = (k,k):

![](https://lms.skillfactory.ru/assets/courseware/v1/7c5025940975cb81148929005d8dfe5e/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_15.png)

Давайте разберём представленную выше формулу на простом примере. Но сначала нас вновь будут ждать довольно сложные формулы — не пугайтесь.

Пример № 1

Найти выборочную корреляцию факторов:

![](https://lms.skillfactory.ru/assets/courseware/v1/64694adccb1ace615100228420e63845/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_16.png)

Смотрим на формулу для выборочной корреляции. Чтобы вычислить коэффициент корреляции c12, необходимо предварительно вычислить x1mean и x2mean — средние значения координат векторов.

Мы уже вычисляли их ранее:

![](https://lms.skillfactory.ru/assets/courseware/v1/a6655c1b4001ddda5f07fca742039bf3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_17.png)

Далее нужно вычислить числитель:

![](https://lms.skillfactory.ru/assets/courseware/v1/6ada8b8a6da887327878d01993ed127a/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_18.png)

Если присмотреться, можно заметить не что иное, как скалярное произведение векторов (x1-x1mean) и (x2-x2mean). Считаем:

![](https://lms.skillfactory.ru/assets/courseware/v1/78925ad0f36b1a82d15a62f682f0a059/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_19.png)

Кажется, мы это уже где-то видели. Да — это векторы x1cent и x2cent, которые мы получили, когда стандартизировали векторы. Посчитаем их скалярное произведение:

![](https://lms.skillfactory.ru/assets/courseware/v1/3b9787f952f2b515ab9d324c90300871/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_20.png)

А что в знаменателе?

![](https://lms.skillfactory.ru/assets/courseware/v1/5e3fa77b5e9840899c93b5057c8e4ac4/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_21.png)

Это произведение длин векторов x1cent и x2cent.

![](https://lms.skillfactory.ru/assets/courseware/v1/a5644ce19365c79e89b4dfc9ab50ec4d/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_22.png)

Мы также уже считали их в примере по стандартизации:

![](https://lms.skillfactory.ru/assets/courseware/v1/26a98091216e3764b944108cd72430f6/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_23.png)

Считаем коэффициент корреляции:

![](https://lms.skillfactory.ru/assets/courseware/v1/122346e6575847f6e6a5b329909b4f7f/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_24.png)

Снова знакомые числа. Да — это элемент на побочной диагонали матрицы Грама, вычисленной для стандартизированных векторов x1cent и x2cent, а значит:

![](data/45.PNG)
***
*Если посчитать корреляцию в обратном порядке между факторами c21 = (x2st, x1st), получим то же самое число, ведь скалярное произведение перестановочно: c12 = c21.*

*Ещё один очевидный факт → Корреляция фактора с самим собой всегда равна 1: cii = 1, то есть c11 = c22 = 1. Так происходит потому, что скалярное произведение вектора с самим собой всегда даёт 1 по свойствам скалярного произведения.*
***

Вот мы и нашли нашу матрицу корреляций:

![](https://lms.skillfactory.ru/assets/courseware/v1/fe38a8cfceea17039b54d7a668139a79/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_25.png)

Она в точности совпадает с матрицей Грама, вычисленной для стандартизированных векторов x1st и x2st:

![](data/46.PNG)

**Геометрическая интерпретация корреляции**

Это косинус угла между центрированными векторами xicent и xjcent. По свойству скалярного произведения:

![](https://lms.skillfactory.ru/assets/courseware/v1/b8a0bb72498df265b1163785689d541f/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_26.png)

**Решение на PYTHON**

В NumPy матрица корреляций вычисляется функцией **np.corrcoef()**:

Получили тот же результат, что и раньше.

В Pandas матрица корреляций вычисляется методом **corr()**, вызванным от имени DataFrame.

In [14]:
x_1 = np.array([1, 2, 6])
x_2 = np.array([3000, 1000, 2000])
np.corrcoef(x_1, x_2)

array([[ 1.        , -0.18898224],
       [-0.18898224,  1.        ]])

На практике корреляция с точки зрения линейной алгебры означает следующее:

* Если корреляция cij = **1**, значит векторы xi и xj **пропорциональны и сонаправлены**.
* Если корреляция cij = **-1**, значит векторы xi и xj п**ропорциональны и противонаправлены**.
* Если корреляция cij = **0**, значит векторы xi и xj **ортогональны друг другу** и, таким образом, являются **линейно независимыми**.

Во всех остальных случаях между факторами xi и xj существует какая-то линейная взаимосвязь, причём чем ближе модуль коэффициента корреляции к 1, тем сильнее эта взаимосвязь. Вспомним классификацию связей факторов, которую мы рассматривали в модуле «EDA-2. Математическая статистика в контексте EDA»:

![](data/47.PNG)

***
Промежуточный вывод ↓  
Таким образом, **матрица корреляций — это матрица Грама**, составленная для стандартизированных столбцов исходной матрицы наблюдений A. Она всегда (в теории) симметричная. На главной диагонали этой матрицы стоят 1, а на местах всех остальных элементов — коэффициенты корреляции между факторами xi и xj.

Если коэффициент корреляции больше 0, то взаимосвязь между факторами прямая (растёт один — растёт второй), в противном случае — обратная (растёт один — падает второй).
***

![](data/48.PNG)

### Пример № 3

Давайте посмотрим на корреляционную матрицу в задаче прогнозирования количества показов квартир агентства недвижимости «Рай в шалаше» в зависимости от разных параметров.

Здесь:

* Demo 2 w — количество показов квартир за две недели;
* Rub — стоимость аренды в рублях;
* Area — площадь квартиры;
* Liv.Area — жилая площадь квартиры;
* Floor — этаж;
* Euro — стоимость аренды в евро;
* NLiv.Area — нежилая площадь квартиры.

![](https://lms.skillfactory.ru/assets/courseware/v1/bb23d029a87a3d7bea81868f09ea72ad/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_28.png)

Матрица получилась размером 7x7, однако её ранг равен 5, а определитель — и вовсе 0. Что это значит? Для начала заметим, что по главной диагонали матрицы стоят единицы — это корреляция каждого фактора с самим собой. Разумеется, матрица симметрична: в первой строке и первом столбце расположены корреляции целевого параметра, то есть количества показов со всеми остальными факторами. Чем эти корреляции больше, тем сильнее взаимосвязь факторов.

Подозрительно одинаковыми выглядят корреляции со стоимостью аренды в рублях и евро. Корреляция между ними равна 1. Это логично так как факторы пропорциональны с каким-то коэффициентом. Кроме того, также странно велика корреляция между жилой и общей площадью. Чистой пропорциональности здесь нет, но из предыдущего модуля мы помним, что жилая, нежилая и общая площади линейно зависимы.

Обратите внимание, что корреляции с нежилой площадью не так велики. Итого мы нашли два избыточных набора факторов: один набор пропорционален, другой просто линейно зависим. Это случай чистой коллинеарности. Уберём по одному фактору из каждого, и ранг станет максимальным. 

Нежилая площадь имеет самую маленькую корреляцию с целевым параметром, поэтому мы избавимся от неё.

Между рублями и евро нет разницы — оставим рубли, так как они нам привычнее.

![](https://lms.skillfactory.ru/assets/courseware/v1/ec9dfbd4d4053051f434998bc27a1e8f/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_29.png)

Итак, мы избавились от нежилой площади и аренды в евро. Ранг стал максимальным (то есть равным 5), чистой коллинеарности больше нет, но определитель всё равно маловат. В чём же дело?

Стоимость аренды жилой площади и общей площади сильно коррелируют между собой. Обратите внимание на значения коэффициентов корреляции — они практически равны 1, хотя формально эти факторы линейно независимы. Такие корреляции ощутимо портят картину, что и отражается на определителе.

Давайте оставим только жилую площадь, её корреляция с показами максимальна.

Корреляции между жилой площадью и этажом уже не такие сильные.

![](https://lms.skillfactory.ru/assets/courseware/v1/1aaa706a52c33da7f8fd3e77db3ff27d/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_4_30.png)

Ранг матрицы теперь равен 3 (как ему и положено), а определитель не так близок к нулю.

Резюмируем ↓

![](data/49.PNG)

### **КАК ОБНАРУЖИТЬ МУЛЬТИКОЛЛИНЕАРНОСТЬ?**

* Иногда видно сразу или заметно по **контексту**, что некоторые факторы будут коррелировать между собой.
* Также можно посмотреть на **определитель** матрицы корреляции: если он близок к нулю, значит дела обстоят не очень хорошо.
* Важным маркером будут странные результаты стандартной регрессионной формулы, например слишком большие по модулю **коэффициенты** (вспомните модуль «ML-2. Обучение с учителем: регрессия», где у нас получились запредельные коэффициенты при решении задач) или взаимно обратные коэффициенты (как мы видели в примере в предыдущем юните). 
* И, наконец, исследование **спектра** матрицы корреляций и числа обусловленности не только позволяет обнаружить мультиколлинераность, но и помогает избавиться от неё.
***
Есть много способов борьбы с мультиколлинеарностью. Мы с вами применили самый наивный — удаление взаимных факторов «на глаз». Увы, это получается не всегда.

Два других метода называются по-разному, но по сути делают одно и тоже: это **метод главных компонент** для корреляционной матрицы и **сингулярное разложение** матрицы факторов. О них мы поговорим в следующем модуле. 

Кроме того, можно воспользоваться знакомыми нам методами **регуляризации**, о которых поговорим уже в этом модуле.

### **В ЧЁМ ПРОБЛЕМА МУЛЬТИКОЛЛИНЕАРНОСТИ ДЛЯ LINEAR REGRESSION?**

Несмотря на то что мультиколлинеарность делает матрицу корреляций более вырожденной, она не оказывает прямого влияния на точность модели сама по себе. Проблема полной вырожденности матрицы (A.T*A), как мы уже обсуждали ранее, в sklearn вполне решается с помощью сингулярного разложения. То есть решение можно получить всегда даже при полной коллинеарности и сильной мультиколлинеарности, несмотря на противоречие с теорией линейной алгебры.


Однако сможем ли мы доверять такому решению?

Бывают задачи, где важно не просто построить модель, но и проинтерпретировать её результат — коэффициенты линейной регрессии. Типичный пример — задача кредитного скоринга: в ней важно понять, что влияет на вероятность дефолта заёмщика.

***

Проблема заключается в том, что в случае мультиколлинеарности коэффициенты линейной регрессии становятся неустойчивыми. Например, признак «остаток долга/сумма выдачи» вроде бы должен приводить к уменьшению вероятности дефолта, так как клиенту остаётся выплачивать всё меньшую сумму. Однако мультиколлинеарность приводит к тому, что подобранный в ходе обучения модели коэффициент может сменить знак на противоположный, а признак, с точки зрения модели, может начать говорить об обратном: чем меньше остаётся платить, тем больше вероятность дефолта. Подобный кейс хорошо описан в [**этой**](https://habr.com/ru/company/akbarsdigital/blog/592493/) статье — рекомендуем с ней ознакомиться.

К тому же, чем больше в данных мультиколлинеарных факторов, тем сильнее увеличивается разброс коэффициентов регрессии. Полная коллинеарность означает, что существует бесконечное количество способов выразить один фактор через линейную комбинацию других. В свою очередь это значит, что есть бесконечное число возможных коэффициентов регрессии , таких, которые дают одни и те же результаты. 

Чем больше высококоррелированных факторов в данных, тем больше таких линейных комбинаций и тем больше коэффициенты становятся по модулю, что приводит к проблеме под названием «взрывной рост весов», когда коэффициенты регрессии начинают стремиться к бесконечности, что приводит к «поломке» даже устойчивой к вырожденным матрицам модели.

In [18]:
# Вычислите коэффициент корреляции между векторами v = (5,1,2) и u = (4,2,8).

x_1 = np.array([5, 1, 2])
x_2 = np.array([4, 2, 8])
np.corrcoef(x_1, x_2).round(2)

array([[1.  , 0.05],
       [0.05, 1.  ]])

Составьте корреляционную матрицу для системы векторов:

x1 = 5.1,1.8,2.1,10.3,12.1,12.6  
x2 = 10.2,3.7,4.1,20.5,24.2,24.1  
x3 = 2.5,0.9,1.1,5.1,6.1,6.3  

Для расчёта используйте библиотеку NumPy или Pandas.

In [37]:
# Чему равен ранг полученной корреляционной матрицы?

x1 = np.array([5.1,1.8,2.1,10.3,12.1,12.6])
x2 = np.array([10.2,3.7,4.1,20.5,24.2,24.1])
x3 = np.array([2.5,0.9,1.1,5.1,6.1,6.3])
A_corr = np.corrcoef([x1, x2,x3])
np.linalg.matrix_rank(A_corr)

3

In [41]:
# Чему равен определитель полученной корреляционной матрицы?
# Ответ округлите до седьмого знака после точки-разделителя.

np.linalg.det(A_corr).round(7)

5e-07